In [1]:
%load_ext autoreload
%autoreload 2

with open("./training_results_v2/v2_repr_contrastive/genes.txt") as f:
    genes: list = f.read().split("\n")

from render_3d_deconv import ValidationRenderer

In [2]:
def drawPieMarker(xs, ys, ratios, sizes, colors, pow_=1, use_max=False, reduce_factor=2.):
    markers = []
    previous = 0
    if use_max:
        ratios=(ratios==ratios.max()).astype(float)
    else:
        ratios[ratios<=sorted(ratios)[-3]]=0#np.quantile(ratios,0.7)
        ratios=ratios**pow_
        ratios=ratios/sum(ratios)
        
    for color, ratio in zip(colors, ratios):
        if ratio>0:
            this = 2 * np.pi * ratio + previous
            x  = [0] + np.cos(np.linspace(previous, this, 90)).tolist() + [0]
            y  = [0] + np.sin(np.linspace(previous, this, 90)).tolist() + [0]
            xy = np.column_stack([x, y])
            previous = this
            markers.append({'marker':xy, 's':np.abs(xy).max()**2*np.array(sizes) / reduce_factor, 'facecolor':color})

    # scatter each of the pie pieces to create pies
    for marker in markers:
        plt.scatter(xs, ys, **marker)
        
def plt_savefig_to_np():
    import io
    import PIL.Image
    buf = io.BytesIO()
    plt.savefig(buf)
    buf.seek(0)
    return np.array(PIL.Image.open(buf)) / 255.0

In [3]:
renderers = {}

In [4]:
# https://giotto-ai.github.io/gtda-docs/latest/notebooks/tmp/mapper_quickstart.html?highlight=mapper

import gc
import gtda
from gtda.mapper.filter import Projection
from gtda.mapper.cover import CubicalCover
from sklearn.cluster import DBSCAN
from create_aligned_umap import create_umap_embeddings
from hdbscan.flat import HDBSCAN_flat
from hdbscan import HDBSCAN
import seaborn as sns
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# import pickle
# import cv2

# with open("./input_data/all_visium_data/preprocessed/autostainer_40x.pkl", "rb") as f:
#     autostainer = pickle.load(f)

# autostainer_thumbnail = cv2.imread("figures/thumbnail_cache/autostainer/thumbnail_32.png")
# ox = 768
# oy = 768

In [19]:
def run_cluster_pipeline(patient_id):
    gc.collect()
    if patient_id == 'autostainer':
        slide_id = 'autostainer'
        pid = ''
    else:
        slide_id, pid = patient_id.split("_")
    
    # Sort of just used as a preprocessor at this point
    if patient_id not in renderers:
        renderers[patient_id] = ValidationRenderer(slide_id, pid, 'figures/v7/' + patient_id)
    renderer = renderers[patient_id]
    
    print("Loaded slide")
    
    true_embed, pred_embed = create_umap_embeddings(renderer.true_visium, renderer.pred_visium)
    
    print("Created UMAP embeddings")
    
    # Configure parallelism of clustering step
    n_jobs = 1

    cover = CubicalCover()
    filter_func = Projection([0, 1])
    clusterer = DBSCAN()

    # Initialise pipeline
    pipe = gtda.mapper.make_mapper_pipeline(
        filter_func=filter_func,
        cover=cover,
        clusterer=clusterer,
        verbose=False,
        n_jobs=n_jobs,
    )
    
    G = pipe.fit_transform(true_embed)
    
    # tunable parameters depending on your slide
    hf = HDBSCAN_flat(true_embed, n_clusters=5, min_cluster_size=120)
#     hf = HDBSCAN(n_clusters=5)
#     hf.fit(true_embed)
    
    print("Created HDBSCAN clusters")
    
    s = []
    c = []
    pal = sns.color_palette()
    embeddings = [true_embed, pred_embed]
#     cl_ = hf.labels_
    # ensuring that every point is colored
    cl_ = hf.labels_ + 1
    colors_ = sns.color_palette(None,cl_.max()+1)
    classes_ = np.eye(cl_.max()+1)
    
    print(patient_id, "HDBSCAN clusters")
    plt.rcParams['figure.figsize'] = (25, 25)
    plt.clf()
    # crop blank_image to spots
    min_x, max_x = renderer.visium_x[include_idxs].min(), renderer.visium_x[include_idxs].max()
    min_y, max_y = renderer.visium_y[include_idxs].min(), renderer.visium_y[include_idxs].max()
    cropped_blank_image = renderer.blank_image[min_y - 256:max_y + 256, min_x - 256:max_x + 256]
    # Get the blank image
    plt.imshow(cropped_blank_image)
    plt.show()
    # Get the cluster-rendered image
    plt.imshow(cropped_blank_image)
    include_idxs = cl_ != -1
    plt.scatter(
        renderer.visium_x[include_idxs],
        renderer.visium_y[include_idxs],
        marker='h',
        # This needs to change to 125 for non autostainer slides
        s=250 if slide_id == 'autostainer' else 125,
        c=[colors_[i] for i in cl_[include_idxs]]
    )
    plt.axis('off')
    plt.show()
    
    for embed_idx in [0, 1]:
        xs,ys,sizes,ratios,colors=[],[],[],[],[]
        
        # G.vs['node_elements'] -> list of ndarray corresponding to indexes
        for NE in G.vs['node_elements']:
            xy=embeddings[embed_idx][NE].mean(0)
            cl_tmp=cl_[NE]
            counts_=classes_[cl_tmp[cl_tmp!=-1]].sum(0)
            size_=counts_.sum()

            if size_>0:
                ratios_=counts_/size_
                xs.append(xy[0])
                ys.append(xy[1])
                sizes.append(int(size_))
                ratios.append(ratios_)
                colors.append(list(colors_))

        plt.rcParams['figure.figsize'] = (6, 5)
        plt.clf()
        for x,y,ratios_,size_,colors_ in zip(xs, ys, ratios, sizes, colors):
            drawPieMarker(x, y, ratios_, size_, colors_, use_max=True,reduce_factor=0.5, pow_=0.7)
        sns.despine()
        plt.axis('off')
        print(patient_id, 'mapper_' + ('true', 'pred')[embed_idx])
        renderer.export(plt_savefig_to_np(), 'mapper_' + ('true', 'pred')[embed_idx])
        plt.show()


In [20]:
patient_ids = [
    'autostainer',
    '092534_24',
    '092534_35',
    '091759_4',
    '091759_7',
    '092146_33',
    '092146_3',
    '092842_17',
    '092842_16',
]

run_cluster_pipeline('092146_33')

/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/spatial_omics/cell-detection/render_3d_deconv.py:274: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes = [torch.tensor(self.cell_detections[i]['boxes']) * 2 for i in self.validation_data['prediction_indices']]
/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/spatial_omics/cell-detection/render_3d_deconv.py:288: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pred_expr_per_cell = [torch.tensor(e) for e in self.validation_data['predictions']['cell_vectors_batch']]


Loaded slide
Created UMAP embeddings
Created HDBSCAN clusters
092146_33 HDBSCAN clusters


/dartfs-hpc/rc/home/x/f006jnx/anaconda3/envs/pyg/lib/python3.8/site-packages/hdbscan/flat.py:155: UserWarning: Cannot predict more than 3 with cluster selection method 'eom'. Changing to method 'leaf'...
  warn(f"Cannot predict more than {max_eom_clusters} with cluster "


UnboundLocalError: local variable 'include_idxs' referenced before assignment

<Figure size 2500x2500 with 0 Axes>